In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import random

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

In [20]:
def get_courses(base_url):
    response = requests.get(base_url, headers=headers)
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')

    # print(name.text[47:-32])
    courses =[]
    # name = parsed_content.find("h1", {"class": "jsx-1970226350 text-white font-weight-bolder mt-0 mb-1 text-lg"})
    # courses.update({"name":name.text[47:-32]})

    main_container = parsed_content.find("div", {"class": "jsx-3749487892 jsx-2822574162 col-9"})
    for index, details in enumerate(main_container.find_all('div', {"class": "jsx-1440666465 jsx-2133140133 card border-0 course-card page_center_body_listing bg-white py-0 pt-3 pb-3 px-2 mx-0 my-2 avatar-container"})):
        ind_course = {}
        c_name = details.find("a", {"class": "jsx-1440666465 jsx-2133140133"})
        exams = details.find_all("a", {"class": "jsx-1440666465 jsx-2133140133 text-sidebar-heading"})
        score = details.find_all("span", {"class": "jsx-1440666465 jsx-2133140133 exam-score text-primary font-weight-bold"})
        extra = details.find_all("span", {"class": "jsx-1440666465 jsx-2133140133"}) # 0,1,2
        # fee = details.find("span", {"class":"jsx-1440666465 jsx-2133140133 fees font-weight-bolder mb-1"})
        ind_course.update({"course":c_name.text})
        for i,t in enumerate(extra[:2]):
            ind_course.update({"detail"  + str(i+1):t.text})
        for e,s in zip(exams,score):
            ind_course.update({e.text:s.text})
        courses.append(ind_course)
    return courses


In [11]:
def get_courses_content(base_url):
    response = requests.get(base_url, headers=headers)
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    print(parsed_content.prettify())


In [25]:
def get_uni():
    with open('main.html', 'r') as f:
        file = f.read()
        file.replace('\n','')
        cleaned_string = re.sub(r'\s+', ' ', file)
        
    parsed_content = BeautifulSoup(cleaned_string, 'html.parser')
    
    unis = []
    for para in parsed_content.find_all("tr", {"class": "jsx-3255989801 automate_client_img_snippet"}):
        college = {}
        rank = para.find("td", {"class": "jsx-3255989801 font-weight-medium text-lg"})
        name = para.find("span", {"class": "jsx-3255989801 w-100 position-relative d-flex align-items-start h-100"})
        state = para.find("span", {"class": "jsx-3255989801 text-gray text-md mr-2 text-capitalize"})
        # course = para.find('a', {'class': 'jsx-3255989801 text-black font-weight-medium text-lg mb-2'})
        acceptance = para.find_all("span", {"class": "jsx-3255989801 bg-gray font-weight-bold text-black mr-1 rounded-sm"})
        span_texts = [span.text.strip() if span else 'NA' for span in acceptance]
        other = para.find("a",{'data-csm-title':"Programs"})
        # fee = para.find("span", {"class": "jsx-3255989801 fees text-capitalize text-success text-lg font-weight-bold"})
        
        if span_texts:
            acceptance = span_texts[0][:-1]
        else:
            acceptance =  ''
        college.update({
                "rank":rank.text[1:],
                "name":name.a['data-csm-title'],
                "state":state.text,
                # "primary_course":course.span['title'],
                "acceptance rate": acceptance,
            })
        # details.update({"fee":fee.text, "other programs": "https://collegedunia.com" + other['href']}) if (fee.text) else details.update({"fee":"N/A", "other programs": "https://collegedunia.com" + other['href']})
        # print("adding-> " + college['name'])
        other_programs = get_courses("https://collegedunia.com" + other['href'])
        for ind_program in other_programs:
            print("adding-> " + college['name'] + " - " + ind_program['course'])
            program = {}
            program.update(college)
            program.update(ind_program)
            unis.append(program)
        # college.update(other_programs)
        # unis.append(details)
        time.sleep(random.randint(0,2))
        
    return unis


In [27]:
universities = get_uni()

adding-> Stanford University, Stanford - Master of Science [M.S] Statistics - Data Science
adding-> Stanford University, Stanford - Bachelor of Science [B.S] Computer Science - Artificial Intelligence
adding-> Stanford University, Stanford - Master of Science [M.Sc] Computer Science - Artificial Intelligence
adding-> Massachusetts Institute of Technology, Cambridge - Master of Science [M.S] Business Analytics
adding-> Massachusetts Institute of Technology, Cambridge - Bachelor of Science [B.S] Computer Science, Economics, and Data Science
adding-> Massachusetts Institute of Technology, Cambridge - Bachelor of Science [B.S] Business Analytics
adding-> Harvard University, Cambridge - Master of Science [M.S] Data Science
adding-> Harvard University, Cambridge - Master of Science [M.S] Health Data Science
adding-> Harvard University, Cambridge - Master Liberal Arts - Data Science
adding-> University of California, Berkeley - Master of  Analytics 
adding-> University of California, Berkeley

In [28]:
df = pd.DataFrame(universities)
df

,rank,name,state,acceptance rate,course,detail1,detail2,IELTS,TOEFL,PTE,GRE,Duolingo,SAT,GMAT
0,1,"Stanford University, Stanford","California, USA",4.8,Master of Science [M.S] Statistics - Data Science,2 years,On Campus,7.0,100,68,328,NaN,NaN,NaN
1,1,"Stanford University, Stanford","California, USA",4.8,Bachelor of Science [B.S] Computer Science - A...,4 years,On Campus,6.5,100,68,NaN,115,NaN,NaN
2,1,"Stanford University, Stanford","California, USA",4.8,Master of Science [M.Sc] Computer Science - Ar...,2 years,On Campus,7,100,68,328,NaN,NaN,NaN
3,2,"Massachusetts Institute of Technology, Cambridge","Massachusetts, USA",8.45,Master of Science [M.S] Business Analytics,12 months,On Campus,7.0,100,NaN,NaN,NaN,NaN,NaN
4,2,"Massachusetts Institute of Technology, Cambridge","Massachusetts, USA",8.45,"Bachelor of Science [B.S] Computer Science, Ec...",4 years,On Campus,NaN,90,NaN,NaN,NaN,1500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,148,"Villanova University, Villanova","Pennsylvania, USA",43.5,Master of Science [M.S] Analytics,2 years,"Online, On Campus",6.5,90,NaN,314,NaN,NaN,NaN
360,149,"University of Massachusetts, North Dartmouth","Massachusetts, USA",,Master of Science [M.S] Data Science,2 years,English,6.5,95,75,297,NaN,NaN,NaN
361,149,"University of Massachusetts, North Dartmouth","Massachusetts, USA",,Bachelor of Science [B.S] Computer Science Ar...,4 years,On Campus,6.5,80,31,NaN,NaN,1370,NaN
362,149,"University of Massachusetts, North Dartmouth","Massachusetts, USA",,Bachelor of Science [B.S] Data Science,4 years,On Campus,6.5,80,31,NaN,NaN,1370,NaN


In [29]:
df.to_excel('Universities_Extended_List.xlsx', index_label=True)